In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import pandas as pd
import time
import csv
import glob
import shutil

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

## Drawing Hands and obtaining coordinates

In [10]:
#create empty list to store all the pd dfs
raw_list = []
processed_list = []

cap = cv2.VideoCapture(0)
start = time.time()

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands = 10) as hands:
    #initialise the frame count
    frame_counter = 0   
    while cap.isOpened():
               
        ret, frame = cap.read()
        
        # count the frame number manually as using cap.get(cv2.CAP_PROP_POS_FRAMES) is tidious
        frame_counter += 1
               
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
                
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
               
        # Rendering results
        # checking if there are any hands in the frame
        if results.multi_hand_landmarks:
                   
            # looping through each feature in the frame
            for num, hand in enumerate(results.multi_hand_landmarks):
                
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
            
            # Appending to the raw list - easier to use for the playback dataset
            raw_list.append(results.multi_hand_landmarks)
            
            # Appending to the processed list - easier to understand data
            
            # list comprehension tuple that contains feature, x, y, z coordinates for a particular frame
            df_frame = [[feature, coordinates.x, coordinates.y, coordinates.z] 
                        for feature, coordinates in enumerate(results.multi_hand_landmarks[0].landmark)]
            
            
            # obtain whether the hand is left or right
            if results.multi_handedness[0].classification[0].label == 'Right':
                right_or_left = 1
            else:
                right_or_left = 0
                
            # measure the amount of time that has elapsed since the programme has started
            # this is to account for the fact that there might be time lags
            end = time.time()
            time_passed = end - start
                            
            # generate a list of the frame count and what hand that is equivalent to the length of the df_frame
            frame_hand_time = [[frame_counter, time_passed, right_or_left] for i in range(len(df_frame))]
            
            # reshape the array so that it's 1d
            convert_np = np.array(frame_hand_time)
            
            #convert to numpy array
            df_frame_np_array = np.array(df_frame)
            
            #concatenate the arrays together
            concate = np.concatenate((convert_np, df_frame_np_array),axis=1)
            
            # create a pd dataframe from the data
            df_appending = pd.DataFrame(concate, columns=['Frame', 'Time_Elapsed', 'Hand', 'Feature', 'x', 'y', 'z'])

            #append to a dataframe list - this will be merged later on
            processed_list.append(df_appending)
            
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### Export Datasets

In [4]:
path = r'D:\Users\Student\OneDrive\Bath\Dissertation\Data\MediaPipe_Implementation'

def exporter(dataset, location, export_name):
    ''' 
    Function that exports the dataset
    Args.
    Dataset - the dataset to export
    Location - the location to export the dataset to
    Export_name - the name of the file saved
    '''
    
    path = location
    dataset.to_csv(os.path.join(path, export_name))

In [5]:
# ### raw data export

# # opening the csv file in 'w+' mode
# file = open(path + '\\'+ 'raw_data.csv', 'w+', newline ='')
  
# # writing the data into the file
# with file:    
#     write = csv.writer(file)
#     write.writerows(raw_list)
    
# ### processed dataset

# merge the dataframe into one set
merged_df = pd.concat(processed_list)

#set the index as the frame
merged_df.set_index('Frame')

exporter(merged_df, path, 'process_data.csv')

## Playback - saves as images

In [13]:
# declare folder to save the images to

path_images = r'D:\Users\Student\OneDrive\Bath\Dissertation\Data\Replay_images'

# empty the folder first - doesn't require access as administrator

for filename in os.listdir(path_images):
    file_path = os.path.join(path_images, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path} Reason: {e}')

# # goes through both hands


## create blank opencv2 matrix - with all 0 values

blank_img = cv2.imread('blank.jpg')
# loop through all the results in the raw list
for i in range(len(raw_list)):
    
    # reinstantiate the blank image for each iteration
    annotated_image = blank_img.copy()
    
    # loop through each hand of i'th element
    for hand in raw_list[i]:
        
        # draw the landmarks on the image
        # takes the image
        # the hand - list of landmarks
        # the connections between the features - mp_hands.HAND_CONNECTIONS
        mp_drawing.draw_landmarks(annotated_image, 
                          hand, mp_hands.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
    name = f'image{i}.jpg'
    cv2.imwrite(os.path.join(path_images, name),annotated_image)

## Playback - save as an video

In [9]:
# need this to sort out the ordering of the files


path_videos = r'D:\Users\Student\OneDrive\Bath\Dissertation\Data\Replay_videos'

 
img_array = []
## this sorts the files in time of modification - glob uses os.listdir to obtain file order, 1, 10, 2
## to fix this use modification time of the files to obtain chronological order 
for filename in sorted(glob.glob(os.path.join(path_images,'*.jpg')), key=os.path.getmtime):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter(os.path.join(path_videos, 'video1.avi'), 0, 8, (width, height))

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()